In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
%matplotlib inline

We're ready to build our first neural network. We will have multiple features we feed into our model, each of which will go through a set of perceptron models to arrive at a response which will be trained to our output.

Like many models we've covered, this can be used as both a regression or classification model.

First, we need to load our dataset. For this example we'll use The Museum of Modern Art in New York's [public dataset](https://media.githubusercontent.com/media/MuseumofModernArt/collection/master/Artworks.csv) on their collection.

In [2]:
artworks = pd.read_csv('https://media.githubusercontent.com/media/MuseumofModernArt/collection/master/Artworks.csv')

In [3]:
artworks.columns

Index(['Title', 'Artist', 'ConstituentID', 'ArtistBio', 'Nationality',
       'BeginDate', 'EndDate', 'Gender', 'Date', 'Medium', 'Dimensions',
       'CreditLine', 'AccessionNumber', 'Classification', 'Department',
       'DateAcquired', 'Cataloged', 'ObjectID', 'URL', 'ThumbnailURL',
       'Circumference (cm)', 'Depth (cm)', 'Diameter (cm)', 'Height (cm)',
       'Length (cm)', 'Weight (kg)', 'Width (cm)', 'Seat Height (cm)',
       'Duration (sec.)'],
      dtype='object')

We'll also do a bit of data processing and cleaning, selecting columns of interest and converting URL's to booleans indicating whether they are present.

In [4]:
# Select Columns.
artworks = artworks[['Artist', 'Nationality', 'Gender', 'Date', 'Department',
                    'DateAcquired', 'URL', 'ThumbnailURL', 'Height (cm)', 'Width (cm)']]

# Convert URL's to booleans.
artworks['URL'] = artworks['URL'].notnull()
artworks['ThumbnailURL'] = artworks['ThumbnailURL'].notnull()

# Drop films and some other tricky rows.
artworks = artworks[artworks['Department']!='Film']
artworks = artworks[artworks['Department']!='Media and Performance Art']
artworks = artworks[artworks['Department']!='Fluxus Collection']

# Drop missing data.
artworks = artworks.dropna()

In [5]:
artworks.head()

,Artist,Nationality,Gender,Date,Department,DateAcquired,URL,ThumbnailURL,Height (cm),Width (cm)
0,Otto Wagner,(Austrian),(Male),1896,Architecture & Design,1996-04-09,True,True,48.6000,168.9000
1,Christian de Portzamparc,(French),(Male),1987,Architecture & Design,1995-01-17,True,True,40.6401,29.8451
2,Emil Hoppe,(Austrian),(Male),1903,Architecture & Design,1997-01-15,True,True,34.3000,31.8000
3,Bernard Tschumi,(),(Male),1980,Architecture & Design,1995-01-17,True,True,50.8000,50.8000
4,Emil Hoppe,(Austrian),(Male),1903,Architecture & Design,1997-01-15,True,True,38.4000,19.1000


## Building a Model

Now, let's see if we can use multi-layer perceptron modeling (or "MLP") to see if we can classify the department a piece should go into using everything but the department name.

Before we import MLP from SKLearn and establish the model we first have to ensure correct typing for our data and do some other cleaning.

In [6]:
# Get data types.
artworks.dtypes

Artist           object
Nationality      object
Gender           object
Date             object
Department       object
DateAcquired     object
URL                bool
ThumbnailURL       bool
Height (cm)     float64
Width (cm)      float64
dtype: object

The `DateAcquired` column is an object. Let's transform that to a datetime object and add a feature for just the year the artwork was acquired.

In [7]:
artworks['DateAcquired'] = pd.to_datetime(artworks.DateAcquired)
artworks['YearAcquired'] = artworks.DateAcquired.dt.year
artworks['YearAcquired'].dtype

dtype('int64')

Great. Let's do some more miscellaneous cleaning.

In [8]:
# Remove multiple nationalities, genders, and artists.
artworks.loc[artworks['Gender'].str.contains('\) \('), 'Gender'] = '\(multiple_persons\)'
artworks.loc[artworks['Nationality'].str.contains('\) \('), 'Nationality'] = '\(multiple_nationalities\)'
artworks.loc[artworks['Artist'].str.contains(','), 'Artist'] = 'Multiple_Artists'

# Convert dates to start date, cutting down number of distinct examples.
artworks['Date'] = pd.Series(artworks.Date.str.extract(
    '([0-9]{4})', expand=False))[:-1]

# Final column drops and NA drop.
X = artworks.drop(['Department', 'DateAcquired', 'Artist', 'Nationality', 'Date'], 1)

# Create dummies separately.
artists = pd.get_dummies(artworks.Artist)
nationalities = pd.get_dummies(artworks.Nationality)
dates = pd.get_dummies(artworks.Date)

# Concat with other variables, but artists slows this wayyyyy down so we'll keep it out for now
X = pd.get_dummies(X, sparse=True)
X = pd.concat([X, nationalities, dates], axis=1)

Y = artworks.Department

In [14]:
# Alright! We've done our prep, let's build the model.
# Neural networks are hugely computationally intensive.
# This may take several minutes to run.

# Import the model.
from sklearn.neural_network import MLPClassifier

# Establish and fit the model, with a single, 1000 perceptron layer.
mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=100, verbose=True, early_stopping=True)
mlp.fit(X, Y)

Iteration 1, loss = 2.11695589
Iteration 2, loss = 0.87796737
Iteration 3, loss = 0.84395796
Iteration 4, loss = 0.82265144
Iteration 5, loss = 0.80879934
Iteration 6, loss = 0.82536076
Iteration 7, loss = 0.78402200
Iteration 8, loss = 0.80868257
Iteration 9, loss = 0.76305248
Iteration 10, loss = 0.77666386
Iteration 11, loss = 0.77579644
Iteration 12, loss = 0.76275963
Iteration 13, loss = 0.79341182
Iteration 14, loss = 0.75405670
Iteration 15, loss = 0.74523381
Iteration 16, loss = 0.75173451
Iteration 17, loss = 0.76310341
Iteration 18, loss = 0.73921373
Iteration 19, loss = 0.76596600
Iteration 20, loss = 0.72533391
Iteration 21, loss = 0.71601340
Iteration 22, loss = 0.73728144
Iteration 23, loss = 0.73251431
Iteration 24, loss = 0.70412393
Iteration 25, loss = 0.70166554
Iteration 26, loss = 0.72611728
Iteration 27, loss = 0.71589523
Iteration 28, loss = 0.71875106
Iteration 29, loss = 0.70310554
Iteration 30, loss = 0.70240710
Iteration 31, loss = 0.72089912
Iteration 32, los

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=100, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=True, warm_start=False)

In [15]:
mlp.score(X, Y)

0.7544759392323799

In [16]:
Y.value_counts()/len(Y)

Drawings & Prints        0.622536
Photography              0.226449
Architecture & Design    0.112957
Painting & Sculpture     0.033686
Media and Performance    0.004372
Name: Department, dtype: float64

In [ ]:
cross_val_score(mlp, X, Y, cv=5)

Now we got a lot of information from all of this. Firstly we can see that the model seems to overfit, though there is still so remaining performance when validated with cross validation. This is a feature of neural networks that aren't given enough data for the number of features present. _Neural networks, in general, like_ a lot _of data_. You may also have noticed something also about neural networks: _they can take a_ long _time to run_. Try increasing the layer size by adding a zero. Feel free to interrupt the kernel if you don't have time...

Also note that we created bools for artist's name but left them out. Both of the above points are the reason for that. It would take much longer to run and it would be much more prone to overfitting.

## Model parameters

Now, before we move on and let you loose with some tasks to work on the model, let's go over the parameters.

We included one parameter: hidden layer size. Remember in the previous lesson, when we talked about layers in a neural network. This tells us how many and how big to make our layers. Pass in a tuple that specifies each layer's size. Our network is 1000 neurons wide and one layer. (100, 4, ) would create a network with two layers, one 100 wide and the other 4.

How many layers to include is determined by two things: computational resources and cross validation searching for convergence. It's generally less than the number of input variables you have.

You can also set an alpha. Neural networks like this use a regularization parameter that penalizes large coefficients just like we discussed in the advanced regression section. Alpha scales that penalty.

Lastly, we'll discuss the activation function. The activation function determines whether the output from an individual perceptron is binary or continuous. By default this is a 'relu', or 'rectified linear unit function' function. In the exercise we went through earlier we used this binary function, but we discussed the _sigmoid_ as a reasonable alternative. The _sigmoid_ (called 'logistic' by SKLearn because it's a 'logistic sigmoid function') allows for continuous variables between 0 and 1, which allows for a more nuanced model. It does come at the cost of increased computational complexity.

If you want to learn more about these, study [activation functions](https://en.wikipedia.org/wiki/Activation_function) and [multilayer perceptrons](https://en.wikipedia.org/wiki/Multilayer_perceptron). The [Deep Learning](http://www.deeplearningbook.org/) book referenced earlier goes into great detail on the linear algebra involved.

You could also just test the models with cross validation. Unless neural networks are your specialty cross validation should be sufficient.

For the other parameters and their defaults, check out the [MLPClassifier documentation](http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html#sklearn.neural_network.MLPClassifier).

## Drill: Playing with layers

Now it's your turn. Using the space below, experiment with different hidden layer structures. You can try this on a subset of the data to improve runtime. See how things vary. See what seems to matter the most. Feel free to manipulate other parameters as well. It may also be beneficial to do some real feature selection work...

In [18]:
mlp = MLPClassifier(hidden_layer_sizes=(100,), learning_rate_init=0.01, max_iter=100, verbose=True, early_stopping=True)
mlp.fit(X, Y)
mlp.score(X, Y)

Iteration 1, loss = 3.30692749
Validation score: 0.689725
Iteration 2, loss = 0.94339364
Validation score: 0.721730
Iteration 3, loss = 0.83342647
Validation score: 0.659749
Iteration 4, loss = 0.76812586
Validation score: 0.733075
Iteration 5, loss = 0.73061356
Validation score: 0.732614
Iteration 6, loss = 0.71519493
Validation score: 0.724313
Iteration 7, loss = 0.71400666
Validation score: 0.744235
Iteration 8, loss = 0.70927163
Validation score: 0.735842
Iteration 9, loss = 0.70759710
Validation score: 0.720439
Iteration 10, loss = 0.69849858
Validation score: 0.743221
Iteration 11, loss = 0.70031151
Validation score: 0.696089
Iteration 12, loss = 0.70115533
Validation score: 0.695720
Iteration 13, loss = 0.71231638
Validation score: 0.701531
Iteration 14, loss = 0.70723686
Validation score: 0.738978
Iteration 15, loss = 0.71945501
Validation score: 0.747002
Iteration 16, loss = 0.72183813
Validation score: 0.693507
Iteration 17, loss = 0.73288193
Validation score: 0.733260
Iterat

0.7442004187689668

In [21]:
cross_val_score(mlp, X, Y, cv=5)

Iteration 1, loss = 3.91370931
Validation score: 0.678889
Iteration 2, loss = 1.30038979
Validation score: 0.653407
Iteration 3, loss = 0.93733294
Validation score: 0.662285
Iteration 4, loss = 0.80378872
Validation score: 0.711864
Iteration 5, loss = 0.77074265
Validation score: 0.454399
Iteration 6, loss = 0.74764242
Validation score: 0.541335
Iteration 7, loss = 0.71909422
Validation score: 0.715554
Iteration 8, loss = 0.72486061
Validation score: 0.521965
Iteration 9, loss = 0.71110895
Validation score: 0.740228
Iteration 10, loss = 0.70028830
Validation score: 0.708290
Iteration 11, loss = 0.71064627
Validation score: 0.740689
Iteration 12, loss = 0.69217403
Validation score: 0.725124
Iteration 13, loss = 0.70581345
Validation score: 0.687421
Iteration 14, loss = 0.71257651
Validation score: 0.723049
Iteration 15, loss = 0.69587803
Validation score: 0.709443
Iteration 16, loss = 0.69135639
Validation score: 0.723049
Iteration 17, loss = 0.70489109
Validation score: 0.725009
Iterat

array([0.70006917, 0.71313933, 0.67526058, 0.6581035 , 0.56722476])

##### Results: The MLP is overfitting. There is a lot of variance in the cross validation folds.